Stage 3: Python  notebook: Demonstrate simple queries in Python notebooks over your data. In particular, apply sentiment analysis to data in different dates and periods and generate simple plots. Enable search queries over the data.

In [ ]:
import lithops

analysis_folder_name = ''

# Retrieve analysis_folder from Stage 1 notebook
%store -r analysis_folder_name

if len(analysis_folder_name) == 0:
    print('You have not defined any folder yet (use Stage 1 notebook or the \'analysis_folder\' variable directly)')
else:
    bucket_name = lithops.Storage().bucket
    data = lithops.Storage().get_object(bucket_name, analysis_folder_name+'.csv', stream=True)

In [ ]:
import numpy as np
import pandas as pd

pd.set_option("display.precision", 2)
df = pd.read_csv(data, parse_dates=['date'])

In [ ]:
import calplot

calplot.calplot(pd.Series(np.random.randn(len(df.index)), index=df['date']), cmap='YlGn', colorbar=False)

In [ ]:
import plotly.graph_objects as go

df['date_day'] = df['date'].dt.date
grouped = df.groupby('date_day')['sentiment'].value_counts()

x_axis = grouped.index.levels[0]
fig = go.Figure()

for n_stars in grouped.index.levels[1]:
    arr = []
    for day in grouped.index.levels[0]:
        arr.append(grouped.get(day).get(n_stars, default=0))
    fig.add_trace(go.Bar(x=x_axis, y=arr, name=n_stars))

fig.update_layout(barmode='stack')
fig.update_yaxes(type="log")
fig.show()

In [ ]:
df = pd.read_csv('test.csv')
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()
#df.memory_usage()

In [ ]:
# https://github.com/lithops-cloud/lithops/blob/master/docs/file_chunking.md

# TODO: Search queries over the date with SQL???  using a library

# date, geographic location, url, and sentiment analysis at least

# https://python-visualization.github.io/folium/quickstart.html#Getting-Started

# https://altair-viz.github.io/gallery/index.html#case-studies

# https://plotly.com/python/

# https://seaborn.pydata.org/examples/index.html

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px

# This dataframe has 244 lines, but 4 distinct values for `day`
df = px.data.tips()

app = dash.Dash(__name__)

app.layout = html.Div([
    html.P("Names:"),
    dcc.Dropdown(
        id='names', 
        value='day', 
        options=[{'value': x, 'label': x} 
                 for x in ['smoker', 'day', 'time', 'sex']],
        clearable=False
    ),
    html.P("Values:"),
    dcc.Dropdown(
        id='values', 
        value='total_bill', 
        options=[{'value': x, 'label': x} 
                 for x in ['total_bill', 'tip', 'size']],
        clearable=False
    ),
    dcc.Graph(id="pie-chart"),
])

@app.callback(
    Output("pie-chart", "figure"), 
    [Input("names", "value"), 
     Input("values", "value")])
def generate_chart(names, values):
    fig = px.pie(df, values=values, names=names)
    return fig

app.run_server(debug=True, use_reloader=False)

In [ ]:
# TODO: Ideas

# https://dash-gallery.plotly.host/named-entity-recognition/ # DO THIS WITH TRANSFORMERS


# https://dash-gallery.plotly.host/dash-interest-rate/
# https://dash-gallery.plotly.host/Portal/


# https://plotly.com/python/plotly-express/

# https://plotly.com/python/basic-charts/

# https://dash.plotly.com/datatable/filtering
# https://dash.plotly.com/datatable/interactivity

In [ ]:
import dash
from dash.dependencies import Input, Output
import dash_table
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminder2007.csv')

app = dash.Dash(__name__)

app.layout = html.Div([
    dash_table.DataTable(
        id='datatable-interactivity',
        columns=[
            {"name": i, "id": i, "deletable": True, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=True,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="multi",
        row_deletable=True,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Div(id='datatable-interactivity-container')
])

@app.callback(
    Output('datatable-interactivity', 'style_data_conditional'),
    Input('datatable-interactivity', 'selected_columns')
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('datatable-interactivity-container', "children"),
    Input('datatable-interactivity', "derived_virtual_data"),
    Input('datatable-interactivity', "derived_virtual_selected_rows"))
def update_graphs(rows, derived_virtual_selected_rows):
    # When the table is first rendered, `derived_virtual_data` and
    # `derived_virtual_selected_rows` will be `None`. This is due to an
    # idiosyncrasy in Dash (unsupplied properties are always None and Dash
    # calls the dependent callbacks when the component is first rendered).
    # So, if `rows` is `None`, then the component was just rendered
    # and its value will be the same as the component's dataframe.
    # Instead of setting `None` in here, you could also set
    # `derived_virtual_data=df.to_rows('dict')` when you initialize
    # the component.
    if derived_virtual_selected_rows is None:
        derived_virtual_selected_rows = []

    dff = df if rows is None else pd.DataFrame(rows)

    colors = ['#7FDBFF' if i in derived_virtual_selected_rows else '#0074D9'
              for i in range(len(dff))]

    return [
        dcc.Graph(
            id=column,
            figure={
                "data": [
                    {
                        "x": dff["country"],
                        "y": dff[column],
                        "type": "bar",
                        "marker": {"color": colors},
                    }
                ],
                "layout": {
                    "xaxis": {"automargin": True},
                    "yaxis": {
                        "automargin": True,
                        "title": {"text": column}
                    },
                    "height": 250,
                    "margin": {"t": 10, "l": 10, "r": 10},
                },
            },
        )
        # check if column exists - user may have deleted it
        # If `column.deletable=False`, then you don't
        # need to do this check.
        for column in ["pop", "lifeExp", "gdpPercap"] if column in dff
    ]


app.run_server(debug=True, use_reloader=False)

In [ ]:
import plotly.express as px
import numpy as np
df = px.data.gapminder().query("year == 2007")
fig = px.treemap(df, path=[px.Constant('world'), 'continent', 'country'], values='pop',
                  color='lifeExp', hover_data=['iso_alpha'])
fig.show()

In [ ]:
import plotly.express as px
df = px.data.gapminder()
fig = px.choropleth(df, locations="iso_alpha", color="lifeExp", hover_name="country", animation_frame="year", range_color=[20,80])
fig.show()


In [ ]:
import plotly.express as px
df = px.data.wind()
fig = px.bar_polar(df, r="frequency", theta="direction", color="strength", template="plotly_dark",
            color_discrete_sequence= px.colors.sequential.Plasma_r)
fig.show()
